In [34]:
using LinearAlgebra, CSV, Random, Tables

In [3]:
diagm(1:3)

3×3 Array{Int64,2}:
 1  0  0
 0  2  0
 0  0  3

In [4]:
data = CSV.File("../NC-Data.csv") |> Tables.matrix;
# Matrix sqrt: via diagonalization and sqrt eigenvalues
raw_data = sqrt(data)';
n = size(data, 1);
N = n * n;
# All close.
maximum(abs.(raw_data * raw_data .- data))

4.884981308350689e-15

In [23]:
selected_data = [3; 83; 85];
y = -ones(n)
y[selected_data] .= 1
opt_inds = [79; 80; 81; 84];
k = 7

7

In [24]:
y[selected_data]

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

In [6]:
s = maximum(svd(raw_data[:, selected_data]).S)
u = svd(raw_data[:, selected_data]).U[:, 1];
v = svd(raw_data[:, selected_data]).V[:, 1];
D = Matrix(1.0I, 101, 101)[:, selected_data];
s

1.4499823628525759

In [7]:
svd(raw_data[:, selected_data]).S .^ 2

3-element Array{Float64,1}:
 2.102448852583539
 0.8291308235464361
 0.06842032387002067

In [8]:
maximum(eigvals(data))

25.582479205536032

In [9]:
maximum(eigvals(data - raw_data' * u * u' * raw_data))

19.502666782823834

In [13]:
maximum(eigvals(raw_data * (I - raw_data * D * v * v' * D' * raw_data' / s^2) * raw_data))

19.5026667828238

In [14]:
maximum(eigvals((I - raw_data * D * v * v' * D' * raw_data' / s^2) * data))

19.502666782823788

In [15]:
K = 7
localSearchSteps = 2
Sigma = data
function YuanSubset(y)
    beta = Hk(startingEig, K, y)

        for i = 1:localSearchSteps
            beta = Hk(sparseTimes(Sigma,beta), K, y)
        end
        bestObj = (transpose(beta)*sparseTimes(Sigma,beta))[1]

    return bestObj, beta
end

YuanSubset (generic function with 1 method)

In [69]:
function NaiveEig(y)
    selected = findall(y .== 1)
    i = K - size(selected)[1]
    permissible = findall(y .== -1)
    return maximum([
        maximum(svd(raw_data[
            [selected; permissible[randperm(size(permissible)[1])[1:i]]],
            :]).S) ^ 2
        for x in 1:1000
    ])
end
min(s^2, NaiveEig(y))

2.102448852583539

In [72]:
# raw_data variables have unit norm.
dk_column_square_norms = mapslices(norm, raw_data, dims=2) .- (raw_data * u) .^ 2
sort(dk_column_square_norms, dims=1)

101×1 Array{Float64,2}:
 0.07840949118222473
 0.083231804551119
 0.09716119881323693
 0.1071646153518997
 0.23200393802442631
 0.25748323108433635
 0.2826088084450322
 0.2994137798860217
 0.30672839486145065
 0.4130785461893913
 ⋮
 0.9974140694696207
 0.9974810282462534
 0.9975074122838447
 0.9990195804099145
 0.9994710806101498
 0.9994990973643176
 0.9996455377887329
 0.9997988552808758
 0.9999954454602588

In [74]:
2 * sum(sort(dk_column_square_norms, dims=1)[1:3]) / min(s^2, NaiveEig(y))

0.24619147736084804